## Import required packages

In [1]:
import os
import datetime

os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf

from data.abc import ABCPreProcessor, ABCTokenizer
from models.symbolic.rnn import FolkLSTM

## Required files and directories

In [2]:
# Mention the path to the datastore
BASE_DIR = "/home/rithomas/project/AI-Music-Generation-Challenge-2020/"
PROCESSED_ABC_FILENAME = 'processed-abc-files'

#BASE_DIR = "/home/richhiey/Desktop/workspace/projects/AI_Music_Challenge_2020/AI-Music-Generation-Challenge-2020"
#ABC_TFRECORD_DIR = os.path.join("/home/richhiey/Desktop/workspace/projects/AI_Music_Challenge_2020", "tfrecords", "abc")
#PROCESSED_ABC_FILENAME = 'processed-abc-files'

## Load preprocessed dataset

In [3]:
ABC_TFRECORD_DIR = os.path.join("/home/rithomas/cache", "ABC", "6-8-Meter/")
preprocessor = ABCPreProcessor(ABC_TFRECORD_DIR, PROCESSED_ABC_FILENAME)
six_eight_meter_dataset = preprocessor.load_tfrecord_dataset()

DOUBLE_JIGS_TFRECORD_DIR = os.path.join("/home/rithomas/cache", "ABC", "Double-Jigs/")
preprocessor = ABCPreProcessor(ABC_TFRECORD_DIR, PROCESSED_ABC_FILENAME)
double_jigs_dataset = preprocessor.load_tfrecord_dataset()

<MapDataset shapes: ({tune_length: ()}, {input: (None, None), output: (None, None)}), types: ({tune_length: tf.int64}, {input: tf.int64, output: tf.int64})>
<MapDataset shapes: ({tune_length: ()}, {input: (None, None), output: (None, None)}), types: ({tune_length: tf.int64}, {input: tf.int64, output: tf.int64})>


# Folk-LSTM

In [4]:
batch_size = 64
training = True
FOLK_LSTM_DIR = os.path.join(BASE_DIR, 'configs', 'lstm_6_8_meter')

data_dims = preprocessor.get_data_dimensions(ABC_TFRECORD_DIR)
data_dims['batch_size'] = batch_size
dataset = preprocessor.prepare_dataset(six_eight_meter_dataset, batch_size)
print(data_dims)

DEFAULT_LR_CONFIG = {
    'initial_lr': 0.005,
    'final_lr': 0.00005,
    'decay_steps': 100000,
}
model = FolkLSTM(FOLK_LSTM_DIR, data_dims, ABC_TFRECORD_DIR, training, DEFAULT_LR_CONFIG)
print(model.get_configs())


## TRAINING PHASE
print('========== TRAINING PHASE ===========')
TRAIN_CONFIG = {
    'print_outputs_frequency': 100,
    'save_frequency': 100,
    'validation_freq': 1000,
    'max_steps_for_model': 95000
}
model.train(dataset, TRAIN_CONFIG)

## FINE TUNING PHASE
print('========== FINE TUNING PHASE ===========')
FINE_TUNE_CONFIG = {
    'print_outputs_frequency': 100,
    'save_frequency': 100,
    'validation_freq': 100,
    'max_steps_for_model': 5000
}
dataset = preprocessor.prepare_dataset(double_jigs_dataset, batch_size)
model.train(dataset, FINE_TUNE_CONFIG)

{'max_timesteps': 255, 'tune_vocab_size': 167, 'batch_size': 64}
/home/rithomas/project/AI-Music-Generation-Challenge-2020/configs/lstm_6_8_meter/lstm.json
{'tune_embedding_size': '16', 'rnn': DictWrapper({'num_layers': '3', 'num_units': '512', 'unit_type': 'lstm'})}
Restored from /home/rithomas/project/AI-Music-Generation-Challenge-2020/configs/lstm_6_8_meter/ckpt/ckpt-4
{'tune_embedding_size': '16', 'rnn': DictWrapper({'num_layers': '3', 'num_units': '512', 'unit_type': 'lstm'})}
========== TRAINING PHASE ===========


InternalError: Blas GEMM launch failed : a.shape=(64, 16), b.shape=(16, 2048), m=64, n=2048, k=16 [Op:MatMul]

# Generate 10000 double jigs!

In [ ]:
from datetime import datetime
start = datetime.now()

#n = 5
n = 10000
start_tokens = ['<s>', 'M:6/8']
temperature = 1
model = FolkLSTM(FOLK_LSTM_DIR, data_dims, ABC_TFRECORD_DIR, False)

abc_file = open(os.path.join(ABC_TFRECORD_DIR, "10000_double_jigs.abc"), 'w', buffering=20*(1024**2))

for i in range(n):
    print('-----------------------------------------------------')
    tune = model.complete_tune(start_tokens, temperature)
    formatted_tune = 'X:' + str(i+1) + '\n' + tune[1] + '\n' + tune[2] + '\n' + 'L:1/8' + '\n' + ''.join(tune[3:len(tune)-1])
    print(formatted_tune)
    abc_file.write(formatted_tune + '\n\n')

abc_file.close()